In [1]:
from nltk.tokenize.regexp import RegexpTokenizer

trainSet1 = open("C:/Users/Paulo/Desktop/BCC/3º Período/IA/PROJETO VA2/Training set 1.txt", 'r', encoding = "ISO-8859-1")

categorias1 = []
subcategorias = []
subcategorias1 = []

for line in trainSet1:
    categorias1.append(line.strip().split(':')[0])
    if(line.strip().split(':')[1] not in subcategorias):
        subcategorias.append(line.strip().split(':')[1])
        
for element in subcategorias:
    if(element.strip().split(' ')[0] not in subcategorias1):
        subcategorias1.append(element.strip().split(' ')[0])

print(categorias1)

trainSet1 = open("C:/Users/Paulo/Desktop/BCC/3º Período/IA/PROJETO VA2/Training set 1.txt", 'r', encoding = "ISO-8859-1")

def preprocessamento(pergunta):
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(pergunta)
    
    tokens = [word for word in tokens if word not in categorias1 and word not in subcategorias1]
    
    return ' '.join(tokens)

conjuntoTreinamento1 = []

for pergunta in trainSet1:
    conjuntoTreinamento1.append(preprocessamento(pergunta))
    
trainSet1.close()
trainSet1.close()

['DESC', 'ENTY', 'DESC', 'ENTY', 'ABBR', 'HUM', 'HUM', 'HUM', 'DESC', 'HUM', 'NUM', 'DESC', 'HUM', 'HUM', 'ENTY', 'LOC', 'DESC', 'NUM', 'DESC', 'DESC', 'NUM', 'ENTY', 'HUM', 'DESC', 'DESC', 'ENTY', 'HUM', 'LOC', 'ENTY', 'LOC', 'ABBR', 'ENTY', 'NUM', 'HUM', 'NUM', 'DESC', 'NUM', 'ENTY', 'LOC', 'ENTY', 'DESC', 'ENTY', 'NUM', 'NUM', 'DESC', 'HUM', 'ENTY', 'HUM', 'DESC', 'ENTY', 'LOC', 'DESC', 'HUM', 'DESC', 'HUM', 'DESC', 'ENTY', 'ENTY', 'LOC', 'DESC', 'LOC', 'DESC', 'LOC', 'DESC', 'HUM', 'LOC', 'DESC', 'DESC', 'LOC', 'DESC', 'LOC', 'HUM', 'ENTY', 'DESC', 'HUM', 'DESC', 'HUM', 'ENTY', 'DESC', 'LOC', 'ENTY', 'NUM', 'DESC', 'LOC', 'ENTY', 'NUM', 'LOC', 'ENTY', 'HUM', 'NUM', 'NUM', 'DESC', 'DESC', 'ENTY', 'DESC', 'DESC', 'HUM', 'DESC', 'NUM', 'LOC', 'HUM', 'DESC', 'DESC', 'ENTY', 'HUM', 'HUM', 'DESC', 'LOC', 'ENTY', 'LOC', 'DESC', 'LOC', 'LOC', 'ENTY', 'ENTY', 'ENTY', 'LOC', 'NUM', 'ENTY', 'DESC', 'LOC', 'ENTY', 'DESC', 'NUM', 'DESC', 'DESC', 'DESC', 'HUM', 'HUM', 'NUM', 'ENTY', 'DESC', 'ENT

In [6]:
from nltk.tokenize.regexp import RegexpTokenizer

trainTest = open("C:/Users/Paulo/Desktop/BCC/3º Período/IA/PROJETO VA2/Test set.txt", 'r', encoding = "ISO-8859-1")

categorias1Test = []
subcategoriasTest = []
subcategorias1Test = []

for line in trainTest:
    categorias1Test.append(line.strip().split(':')[0])
    if(line.strip().split(':')[1] not in subcategoriasTest):
        subcategoriasTest.append(line.strip().split(':')[1])
        
for element in subcategoriasTest:
    if(element.strip().split(' ')[0] not in subcategorias1Test):
        subcategorias1Test.append(element.strip().split(' ')[0])

print(categorias1Test)

trainTest = open("C:/Users/Paulo/Desktop/BCC/3º Período/IA/PROJETO VA2/Test set.txt", 'r', encoding = "ISO-8859-1")

def preprocessamento1(pergunta):
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(pergunta)
    
    tokens = [word for word in tokens if word not in categorias1Test and word not in subcategorias1Test]
    
    return ' '.join(tokens)

conjuntoTreinamento1Test = []

for pergunta in trainTest:
    conjuntoTreinamento1Test.append(preprocessamento1(pergunta))
    
trainTest.close()
trainTest.close()

['NUM', 'LOC', 'HUM', 'DESC', 'NUM', 'NUM', 'HUM', 'ENTY', 'DESC', 'DESC', 'LOC', 'HUM', 'NUM', 'HUM', 'NUM', 'NUM', 'ENTY', 'HUM', 'DESC', 'NUM', 'HUM', 'DESC', 'LOC', 'DESC', 'DESC', 'HUM', 'DESC', 'LOC', 'LOC', 'LOC', 'NUM', 'LOC', 'DESC', 'NUM', 'NUM', 'NUM', 'LOC', 'NUM', 'NUM', 'NUM', 'ENTY', 'DESC', 'DESC', 'DESC', 'ENTY', 'ENTY', 'NUM', 'DESC', 'NUM', 'HUM', 'DESC', 'HUM', 'HUM', 'DESC', 'DESC', 'DESC', 'NUM', 'LOC', 'LOC', 'NUM', 'LOC', 'HUM', 'LOC', 'ENTY', 'LOC', 'HUM', 'ENTY', 'DESC', 'NUM', 'LOC', 'NUM', 'NUM', 'LOC', 'HUM', 'LOC', 'DESC', 'ENTY', 'NUM', 'NUM', 'HUM', 'DESC', 'NUM', 'HUM', 'NUM', 'NUM', 'DESC', 'DESC', 'HUM', 'DESC', 'LOC', 'ENTY', 'DESC', 'NUM', 'NUM', 'HUM', 'LOC', 'NUM', 'DESC', 'DESC', 'ENTY', 'DESC', 'HUM', 'DESC', 'HUM', 'LOC', 'DESC', 'NUM', 'ENTY', 'NUM', 'LOC', 'ENTY', 'DESC', 'NUM', 'HUM', 'ENTY', 'HUM', 'NUM', 'ENTY', 'NUM', 'LOC', 'NUM', 'NUM', 'DESC', 'HUM', 'NUM', 'HUM', 'NUM', 'NUM', 'DESC', 'NUM', 'NUM', 'HUM', 'DESC', 'ENTY', 'DESC', 'ENTY

In [18]:
conjuntoTreinamento1Test

['How far is it from Denver to Aspen',
 'What county is Modesto California in',
 'Who was Galileo',
 'What is an atom',
 'When did Hawaii become a',
 'How tall is the Sears Building',
 'George Bush purchased a small interest in which baseball team',
 'What is Australia s national flower',
 'Why does the moon turn orange',
 'What is autism',
 'What had a world fair in 1900',
 'What person s head is on a dime',
 'What is the average of a Yellow Labrador',
 'Who was the first man to fly across the Pacific Ocean',
 'When did Idaho become a',
 'What is the life expectancy for crickets',
 'What metal has the highest melting point',
 'Who developed the vaccination against polio',
 'What is epilepsy',
 'What year did the Titanic sink',
 'Who was the first American to walk in space',
 'What is a biosphere',
 'What river in the US is known as the Big Muddy',
 'What is bipolar disorder',
 'What is cholesterol',
 'Who developed the Macintosh computer',
 'What is caffeine',
 'What imaginary line is

## BOW

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
bow_model  = vectorizer.fit(conjuntoTreinamento1)

X_bow_train = bow_model.transform(conjuntoTreinamento1)
X_bow_test  = bow_model.transform(conjuntoTreinamento1Test)

print(X_bow_train.todense()[0,1:200])

print(X_bow_train.shape,X_bow_test.shape)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
(1000, 2720) (500, 2720)


In [9]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()

In [11]:
clf.fit(X_bow_train, categorias1)

acc = clf.score(X_bow_test , categorias1Test)
print('Acurácia: ', acc)

Acurácia:  0.518


## Tf

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(use_idf=False)
tf_model = vectorizer.fit(conjuntoTreinamento1)

X_tf_train = tf_model.transform(conjuntoTreinamento1)
X_tf_test  = tf_model.transform(conjuntoTreinamento1Test)

print(X_tf_train[0,:])

  (0, 156)	0.3333333333333333
  (0, 758)	0.3333333333333333
  (0, 769)	0.3333333333333333
  (0, 1252)	0.3333333333333333
  (0, 1277)	0.3333333333333333
  (0, 1451)	0.3333333333333333
  (0, 2147)	0.3333333333333333
  (0, 2200)	0.3333333333333333
  (0, 2454)	0.3333333333333333


## Tfidf

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(use_idf=True)
tfidf_model = vectorizer.fit(conjuntoTreinamento1)

X_tfidf_train = tfidf_model.transform(conjuntoTreinamento1)
X_tfidf_test  = tfidf_model.transform(conjuntoTreinamento1Test)

print(X_tfidf_train[0,:])

  (0, 2454)	0.3987893700645933
  (0, 2200)	0.422532658148168
  (0, 2147)	0.3987893700645933
  (0, 1451)	0.422532658148168
  (0, 1277)	0.14890042947248563
  (0, 1252)	0.17763952349600942
  (0, 769)	0.21518061812895703
  (0, 758)	0.422532658148168
  (0, 156)	0.21518061812895703


In [14]:
clf.fit(X_tfidf_train, categorias1)

acc = clf.score(X_tfidf_test , categorias1Test)
print('Acurácia: ', acc)

Acurácia:  0.656


In [11]:
docs = ['dist How far is it from Denver to Aspen ?']
preprocessed_docs = [preprocessamento1(pergunta) for pergunta in docs]

print('\nafter preprocessing:')
print(preprocessed_docs)

docs_preds = clf.predict(tfidf_model.transform(preprocessed_docs))

print('\npredictions:')
for i,pergunta in enumerate(docs):
    print('{} -> {}'.format(doc, categorias1[docs_preds[i]]))


after preprocessing:
['How far is it from Denver to Aspen']

predictions:


NameError: name 'doc' is not defined